# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("cities_table.csv")
cities_df.head(5)

,city_name,long,lat,temp,humidity,cloud,wind
0,Grindavik,-22.4338,63.8424,50.47,100,100,9.22
1,Pangnirtung,-65.7125,66.1451,46.87,93,95,4.27
2,Provideniya,-173.3000,64.3833,49.01,71,48,3.89
3,East London,27.9116,-33.0153,65.34,88,75,5.75
4,Cape Town,18.4232,-33.9258,57.34,83,0,11.50


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
gmaps.configure(api_key= g_key)
locations = cities_df[["lat","long"]]
weights = cities_df["humidity"]
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig
#map1

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:
cities_df[(cities_df["temp"] > 70) & (cities_df["temp"] < 90)]

,city_name,long,lat,temp,humidity,cloud,wind
8,Shimoda,138.9500,34.6667,76.10,74,96,11.81
9,Saint Anthony,-93.2180,45.0205,72.99,72,0,10.36
13,Alofi,-169.9187,-19.0595,76.35,72,4,18.54
18,Coahuayana Viejo,-103.6833,18.7333,79.36,90,99,2.62
19,Richards Bay,32.0377,-28.7830,70.36,73,2,5.32
...,...,...,...,...,...,...,...
520,Orange Cove,-119.3137,36.6244,81.77,42,0,4.61
523,Ilhabela,-45.3581,-23.7781,73.98,58,35,4.29
524,Santa Cruz del Sur,-77.9956,20.7147,80.76,71,1,2.30
527,Mitsamiouli,43.2844,-11.3847,76.59,69,22,5.73


In [46]:
hotel_df = cities_df[(cities_df["temp"] > 70) & (cities_df["temp"] < 90) & (cities_df["humidity"] < 30) & (cities_df["cloud"]==0)& (cities_df["wind"] < 10)]
hotel_df["Hotel Name"] = ""
hotel_df

C:\Users\Justin\AppData\Local\Temp/ipykernel_13796/855472604.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,city_name,long,lat,temp,humidity,cloud,wind,Hotel Name
23,Opuwo,13.8400,-18.0607,71.04,25,0,6.98,
49,Saint George,-113.5841,37.1041,87.48,26,0,6.91,
173,Khāsh,61.2158,28.2211,87.84,8,0,6.40,
179,Tyup,78.3648,42.7276,78.76,23,0,6.76,
239,Yangi Marg`ilon,71.7189,40.4272,84.92,21,0,4.61,
261,Zabul Province,67.2500,32.2500,79.29,15,0,3.85,
331,Yumen,97.2000,40.2833,88.39,6,0,8.88,
424,Zhangye,100.4517,38.9342,86.18,8,0,4.88,
492,Chaman,66.4512,30.9236,88.23,13,0,4.12,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [47]:
for index, hotel in hotel_df.iterrows():
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?keyword=hotel&location={hotel['lat']}%2C{hotel['long']}&radius=1500&type=hotel&key={g_key}"
    response = requests.get(url).json()
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
    except:
        print("no hotel found")
hotel_df

c:\Users\Justin\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


no hotel found
no hotel found
no hotel found


,city_name,long,lat,temp,humidity,cloud,wind,Hotel Name
23,Opuwo,13.8400,-18.0607,71.04,25,0,6.98,Ohakane Guesthouse
49,Saint George,-113.5841,37.1041,87.48,26,0,6.91,Red Lion Hotel & Conference Center St. George
173,Khāsh,61.2158,28.2211,87.84,8,0,6.40,خابگاه شبانه میر بلوچ
179,Tyup,78.3648,42.7276,78.76,23,0,6.76,
239,Yangi Marg`ilon,71.7189,40.4272,84.92,21,0,4.61,DIYOR HOTEL
261,Zabul Province,67.2500,32.2500,79.29,15,0,3.85,
331,Yumen,97.2000,40.2833,88.39,6,0,8.88,
424,Zhangye,100.4517,38.9342,86.18,8,0,4.88,Jiaheng Hotel
492,Chaman,66.4512,30.9236,88.23,13,0,4.12,Al khaleej hotel


In [55]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city_name}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "long"]]

In [56]:
# Add marker layer ontop of heat map
fig.add_layer(gmaps.marker_layer(locations, info_box_content=hotel_info))

# Display figure
fig
#map2

Figure(layout=FigureLayout(height='420px'))